In [1]:
import os
# Find the latest version of spark
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [90.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [903 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports In

In [3]:
# Create Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Review").getOrCreate()

In [4]:
# Load data
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Music_Purchase_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10293599|R14LVB34Z2Z53I|B000YMOQZY|     262935067|Knockin' On Heave...|Digital_Music_Pur...|          5|            0|          0|   N|                Y|           favorites|Great  rendition....| 2015-08-31|
|         US|    6087195|R2M0R9CWPC82LT|B00ISAEC80|     452318038|            Flawless|Digital_Music_Pur

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R14LVB34Z2Z53I|          5|            0|          0|   N|                Y|
|R2M0R9CWPC82LT|          5|            0|          0|   N|                Y|
| RQNQI2R3BM52A|          4|            0|          0|   N|                Y|
| RTUIX1135HUAB|          5|            0|          0|   N|                Y|
| RE3QSAT8TL010|          4|            0|          0|   N|                Y|
|R14TN65IY0NUOD|          5|            0|          0|   N|                Y|
|R3LIF8MCNH1ODF|          5|            0|          0|   N|                Y|
|R33X0DGA4PPQ3L|          5|            3|          3|   N|                N|
|R2DJJP7DQRT1ZW|          5|            0|          1|   N|                Y|
|R2A8XDXW5XOT4H|          5|            0|          0|   N|     

In [6]:
# Filtered DF of records with 20+ total votes
# Load in SQL fx to use columns
from pyspark.sql.functions import col

filtered_df = vine_df.filter(col("total_votes") > 19)

filtered_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1S0N56H5B9SUX|          5|           30|         30|   N|                Y|
| RCDI7Z8VCZJI9|          4|           40|         43|   N|                N|
|R3F4225VTUO2WE|          1|            8|         33|   N|                N|
|  R3ADPNURCCTK|          1|            1|         39|   N|                N|
|R2LWBYC8TEUVF2|          5|           50|         53|   N|                Y|
|R23E4EIJXID5IC|          5|           43|         43|   N|                N|
|R1WFSWDT8L5RSJ|          4|           23|         26|   N|                N|
|R37LMHN02KOER1|          5|           50|         50|   N|                N|
|R297QI4VU3LMY6|          4|           41|         50|   N|                N|
| RGNU1L1XFX33B|          3|           34|         35|   N|     

In [7]:
# Filtered DF of records with 50% of votes are helpful

half_help_df = filtered_df.filter(col("helpful_votes")/col("total_votes") > 0.49)

half_help_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1S0N56H5B9SUX|          5|           30|         30|   N|                Y|
| RCDI7Z8VCZJI9|          4|           40|         43|   N|                N|
|R2LWBYC8TEUVF2|          5|           50|         53|   N|                Y|
|R23E4EIJXID5IC|          5|           43|         43|   N|                N|
|R1WFSWDT8L5RSJ|          4|           23|         26|   N|                N|
|R37LMHN02KOER1|          5|           50|         50|   N|                N|
|R297QI4VU3LMY6|          4|           41|         50|   N|                N|
| RGNU1L1XFX33B|          3|           34|         35|   N|                N|
|R2NPE00F48X0ET|          5|           22|         27|   N|                N|
| RSPQFPJ4ISKS3|          5|           32|         35|   N|     

In [8]:
# Filtered DF of records with vine program

paid_df = half_help_df.filter(col("vine") == "Y")

paid_df.show()

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [9]:
# Filtered DF of records with vine program

unpaid_df = half_help_df.filter(col("vine") == "N")

unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1S0N56H5B9SUX|          5|           30|         30|   N|                Y|
| RCDI7Z8VCZJI9|          4|           40|         43|   N|                N|
|R2LWBYC8TEUVF2|          5|           50|         53|   N|                Y|
|R23E4EIJXID5IC|          5|           43|         43|   N|                N|
|R1WFSWDT8L5RSJ|          4|           23|         26|   N|                N|
|R37LMHN02KOER1|          5|           50|         50|   N|                N|
|R297QI4VU3LMY6|          4|           41|         50|   N|                N|
| RGNU1L1XFX33B|          3|           34|         35|   N|                N|
|R2NPE00F48X0ET|          5|           22|         27|   N|                N|
| RSPQFPJ4ISKS3|          5|           32|         35|   N|     

In [16]:
# Count of paid reviews
from pyspark.sql.functions import count

total_paid = paid_df.select(count("review_id"))
total_paid.show()

+----------------+
|count(review_id)|
+----------------+
|               0|
+----------------+



In [17]:
# Count of unpaid reviews

total_unpaid = unpaid_df.select(count("review_id"))
total_unpaid.show()

+----------------+
|count(review_id)|
+----------------+
|            4533|
+----------------+



In [19]:
# Count of unpaid 5 star reviews

five_star_unpaid = unpaid_df.filter("star_rating=5").select(count("review_id"))
five_star_unpaid.show()

+----------------+
|count(review_id)|
+----------------+
|            2507|
+----------------+



In [33]:
percent_of_total_unpaid = (five_star_unpaid.collect()[0][0] / total_unpaid.collect()[0][0])
print("{:.2f}%".format(percent_of_total_unpaid))

0.55%


In [35]:
# Paid review DF results
paid_results_df = spark.createDataFrame([(total_paid.collect()[0][0], 0, '')], ["Total Reviews", "5-Star Reviews", "Percent of 5-Star"])
paid_results_df.show()

+-------------+--------------+-----------------+
|Total Reviews|5-Star Reviews|Percent of 5-Star|
+-------------+--------------+-----------------+
|            0|             0|                 |
+-------------+--------------+-----------------+



In [37]:
# Paid review DF results
unpaid_results_df = spark.createDataFrame([(total_unpaid.collect()[0][0], five_star_unpaid.collect()[0][0], percent_of_total_unpaid)], ["Total Reviews", "5-Star Reviews", "Percent of 5-Star"])
unpaid_results_df.show()

+-------------+--------------+------------------+
|Total Reviews|5-Star Reviews| Percent of 5-Star|
+-------------+--------------+------------------+
|         4533|          2507|0.5530553717185087|
+-------------+--------------+------------------+

